<a href="https://colab.research.google.com/github/7triox/colab-kaggle/blob/Colab/NoiseR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@title Step 1 : Installing

# Install the required packages
!pip install pydub
!apt-get install -y ffmpeg

# Import necessary libraries
import os

# Create directories
os.makedirs('audio_input', exist_ok=True)
os.makedirs('audio_output', exist_ok=True)

In [ ]:
#@title Step 2 & 3 : Upload Files, Remove Silence, and Download with Unique Filename

import shutil
import os
from google.colab import files
from pydub import AudioSegment
from pydub.silence import split_on_silence
from datetime import datetime

# Step 2: Upload Files
def clear_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

# Clear and prepare directories
input_directory = 'audio_input'
output_directory = 'audio_output'
clear_directory(input_directory)
clear_directory(output_directory)

# Upload audio files
print("Upload your audio files:")
uploaded = files.upload()
for filename in uploaded.keys():
    with open(os.path.join(input_directory, filename), 'wb') as f:
        f.write(uploaded[filename])
    print(f"Uploaded: {filename}")

# Step 3: Process Files (Remove Silence) and Download
def remove_silence(input_path, output_path, format):
    # Load the audio file
    audio = AudioSegment.from_file(input_path, format=format)

    # Split audio on silences longer than 500ms
    chunks = split_on_silence(
        audio,
        min_silence_len=500,  # Silence threshold in milliseconds
        silence_thresh=-70,   # dB threshold for silence detection
        keep_silence=100      # Keep 100ms of silence between chunks for natural pacing
    )

    # Combine the chunks back together
    output_audio = AudioSegment.empty()
    for chunk in chunks:
        output_audio += chunk

    # Export the processed audio file
    output_audio.export(output_path, format=format)

# Process each uploaded file
for filename in os.listdir(input_directory):
    if filename.endswith('.mp3') or filename.endswith('.wav'):
        input_path = os.path.join(input_directory, filename)
        output_format = filename.split('.')[-1]
        # Generate a unique filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # e.g., 20250223_154532
        base_name = os.path.splitext(filename)[0]
        output_filename = f"{base_name}_cut_{timestamp}.{output_format}"
        output_path = os.path.join(output_directory, output_filename)

        # Remove silence and save
        remove_silence(input_path, output_path, output_format)
        print(f"Processed: {output_filename}")

        # Automatically download the processed file
        print(f"Downloading: {output_filename}")
        files.download(output_path)

print("Done Uploading, Processing, and Downloading")